### Imports and function definitions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import important libraries and download data
import os
import math
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import multiprocessing
import tensorflow as tf
%matplotlib notebook
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from torch.autograd import Variable
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import xgboost as xgb
from tqdm import tqdm
import string
! pip install tqdm boto3 requests regex sentencepiece sacremoses
! git clone https://github.com/Gaurav-Pande/AES_DL.git && mv AES_DL/data .
! pip install transformers
# ! pip install xgboost
! pip install language-tool-python 
import language_tool_python

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 132 kB 8.7 MB/s 
     |████████████████████████████████| 1.2 MB 63.5 MB/s 
     |████████████████████████████████| 895 kB 74.6 MB/s 
     |████████████████████████████████| 8.7 MB 66.5 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |████████████████████████████████| 138 kB 110.8 MB/s 
     |████████████████████████████████| 127 kB 106.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

Cloning into 'AES_DL'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 59 (delta 25), reused 28 (delta 8), pack-reused 0
Unpacking objects: 100% (59/59), done.
     |████████████████████████████████| 4.0 MB 8.4 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
     |████████████████████████████████| 6.6 MB 69.3 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertModel, BertConfig, BertTokenizer

In [ ]:
tool = language_tool_python.LanguageTool('en-US')
def check_sp_n_grammar (text):
  matches = tool.check(text)
  num_sp_err = 0
  num_gram_err = 0
  num_other_err = 0
  # print ("Spell n Grammar checker: Number of errors detected: ",len(matches))
  for i in range(len(matches)):
    if (matches[i].ruleIssueType == "misspelling"):
      num_sp_err = num_sp_err +1
    elif (matches[i].ruleIssueType == "grammar"):
      num_gram_err = num_gram_err +1
    else:
      num_other_err = num_other_err +1
  #if (matches[i].ruleId == '')
  return (matches, num_sp_err, num_gram_err, num_other_err)

# Prepare Input data (1 essay) for prediction
def prepare_input_data(text, max_len=200):
  tokenized_text = tokenizer.encode(text, add_special_tokens=True ,max_length=200)
  # print ("Tokenized text: ", tokenized_text)
  ## processing the tokenized train values for the test set
  padded_text = np.array( [tokenized_text + [0]*(max_len-len(tokenized_text))])
  # print ("Padded text: ", padded_text)
  attention_mask_test = np.where(padded_text != 0, 1, 0)
  input_ids = torch.tensor(padded_text)
  attention_mask = torch.tensor(attention_mask_test)
  #last_hidden_state = torch.zeros(1,200,768)
  outputs = model(input_ids)
  return(outputs[0])

Unzipping /tmp/tmpzuuna0kj.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [ ]:
# Normalizing the domain1_score
def normalize_value(score, min_value, max_value):
  result =  tf.compat.v1.div(float(tf.subtract(score, min_value)), float(tf.subtract(max_value, min_value)))
  return result

In [ ]:
# taking care of NEC
def clean_nec(essay):
    essay = re.sub(r"@[A-Za-z0-9]+", ' ', essay)
    essay = re.sub(r"https?://[A-Za-z0-9./]+", ' ', essay)
    #essay = re.sub(r"[^a-zA-Z.!?']", ' ', essay)
    essay = re.sub(r" +", ' ', essay)
    return essay

In [ ]:
# augmenting for coherence model
def coherence_augment(essay):
  x = re.split('\. |\? |! ', essay)
  random.shuffle(x)
  return '. '.join(x)

In [ ]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [ ]:
def prepare_input_data_sen_avg(model, tokenizer, text, max_len=200):
    lhs = torch.empty(1,max_sentences,768, dtype=torch.float)
    emb_for_padding = tokenizer.encode_plus("", add_special_tokens=True, truncation=True, padding="max_length", return_tensors="pt", max_length=10)
    tt = torch.tensor(emb_for_padding['input_ids'])
    output = model(tt)
    lhs_for_padding = output.hidden_states[12] + output.hidden_states[11] + output.hidden_states[10] + output.hidden_states[9]
    lhs_for_padding_np = np.array(lhs_for_padding.detach().numpy())
    lhs_for_padding_mean = np.mean(lhs_for_padding_np,axis=1)
    lhs_avg_for_padding = torch.tensor(lhs_for_padding_mean[0])
    sentences = re.split('\. |\? |! ', text)
    sen_length = len(sentences)
    lhs_sentence_avg = np.zeros((max_sentences,768), dtype=float)
    for i,s in enumerate(sentences):
        if (i>=max_sentences):
            break
        tokenize_sentence = tokenizer.encode(sentences[i],add_special_tokens=True, max_length=512, truncation=True)
        tt = torch.tensor(tokenize_sentence)
        tts = tt.reshape(1,len(tt))
        output = model(tts)
        lhs_sentence = output.hidden_states[12] + output.hidden_states[11] + output.hidden_states[10] + output.hidden_states[9]
        lhs_sentence_np = np.array(lhs_sentence.detach().numpy())
        lhs_sentence_np_mean = np.mean(lhs_sentence_np,axis=1)
        lhs_sentence_avg[i] = lhs_sentence_np_mean[0]

    lhs[0] = torch.tensor(lhs_sentence_avg)
        
    if (sen_length < max_sentences):
        for i in range (sen_length, max_sentences):
            lhs[0][i]= lhs_avg_for_padding
            
    #print ("SIze of lhs_for_padding: ", lhs_for_padding.shape)
    #print ("SIze of lhs_avg_for_padding: ", lhs_avg_for_padding.shape)
    #print ("SIze of lhs being returned: ", lhs.shape)
    return (lhs)

In [ ]:
#Loading the dataset
dataset_path = "./data/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
data['normalized_score'] = data.apply(lambda x: float(normalize_value(x['domain1_score'], min_scores[x['essay_set']-1], max_scores[x['essay_set']-1])), axis=1)
data['essay'] = data['essay'].apply(lambda x: clean_nec(x))
data.head()

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


From /usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


,essay_id,essay_set,essay,domain1_score,normalized_score
0,1,1,"Dear local newspaper, I think effects computer...",8,0.6
1,2,1,"Dear , I believe that using computers will ben...",9,0.7
2,3,1,"Dear, More and more people use computers, but ...",7,0.5
3,4,1,"Dear Local Newspaper, I have found that many e...",10,0.8
4,5,1,"Dear , I know having computers has a positive ...",8,0.6


In [ ]:
# LSTM Model
from keras.layers import Embedding, Input, LSTM, Dense, Dropout, Lambda, Flatten, Bidirectional, Conv2D, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.models import Sequential,Model, load_model, model_from_config
import keras.backend as K
max_sentences = 128

def get_model(Hidden_dim1=400, Hidden_dim2=128, return_sequences = True, dropout_dense=0.5, dropout_lstm=0.4, recurrent_dropout=0.4, 
              sen_size=max_sentences, input_size=768, activation='sigmoid', opt_engine='rmsprop', loss_fn='mean_squared_error'):
    """Define the model."""
    model = Sequential()
    model.add(LSTM(Hidden_dim1, dropout=dropout_lstm, recurrent_dropout=recurrent_dropout, input_shape=(sen_size,input_size), return_sequences=return_sequences))
    #model.add(LSTM(Hidden_dim1, dropout=dropout_lstm, recurrent_dropout=recurrent_dropout, input_shape=(input_size,sen_size), return_sequences=return_sequences))
    model.add(LSTM(Hidden_dim2, recurrent_dropout=recurrent_dropout))
    model.add(Dropout(dropout_dense))
    model.add(Dense(1, activation=activation))

    model.compile(loss=loss_fn, optimizer=opt_engine, metrics=['mae'])
    model.summary()
    return model

In [ ]:
average_essay_lens = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AES/average_essay_set_lengths.csv')

In [ ]:
experiment_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW/'

In [ ]:
def prepare_embeddings (df, file_name = "default_embedding_file",
                                file_path="./", max_sentences=128, max_words=512, hstate='last4sum', 
                                essay_field="essay",gold_field="normalized_score",get_sets=True ):
  #
  # embedding_type hardcoded to 'sen_avg':  Averages embeddings for every sentence, embedding vector size per 
  #                                   essay: (max_sentences * 768)
  # 
  # Arguments Description:
  # ----------------------
  # 
  #   df:             Dataframe
  #
  #   file_path:      Base directory where models and embeddings will be saved
  #
  #   file_name:      Base filename of saved embedding files (xxx_lhs, xxx_y. ???
  #
  #   max_sentences:  Relevant for sentence average (sen_avg) embedding type: maximum no of sentences 
  #                   permissable in an essay
  #
  #   max_words:      Relevant for the full embedding (full_emb) embedding type: maximum no of words
  #                   permissable in an essay
  #
  #   hstate:         How embedding is computed using BERT's hidden states:
  #                     - 'last4sum': Embedding computed by summing last 4 hidden state sof instantited BERT model
  #                     - 'second_last':  Embedding computed by picking the 2nd last hidden state of instantiaed BERT model
  #
  #   gold_field:     'y' column for the embeddings. Default is normalized_score
  #
  #   get_sets:        Returns a list of touples with lhs and corresponding y values for each set of essays. 
  #                    DON'T SET TO FALSE RIGHT NOW, WILL BREAK
  #
  #   essay_field:     Column with essay text. For prel, set this to 'combined_essay'. Default is 'essay'
    
    print ("Preparing Embeddings...")
    print ("hState: ", hstate)
    print ("Save File Directory: ", file_path)
    print ("Save File Name: ", file_name)
    if (not df.empty):
        print ("Dataframe provided, Size: ", df.shape)
    else:
        print("Empty Dataset!")
        return
    
    lhs_path = file_path + "/" + file_name + '_lhs.pt'
    y_path = file_path + "/" + file_name + '_y.pt'
    sets_lhs_path = file_path + "/" + file_name + "_sets.pt"
    
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
    model = BertModel.from_pretrained('bert-base-uncased', config=config)

    cuda = torch.device('cuda')
    sets = []
    cat_lhs = torch.empty((0,128,768))
    df_sets = [df.loc[df['essay_set']==i] for i in range(1,9)]
    cat_y_gold = pd.Series()
    set_idx = 1
    
    for x_set in df_sets:
        print("Creating embeddings for set: %d" %set_idx)
        set_idx+=1
        print("Shape of the set: ", x_set.shape)
        essays = x_set[essay_field]
        y_gold = x_set[gold_field]
        cat_y_gold = cat_y_gold.append(y_gold)
        print("Concatenated y_gold shape: ",cat_y_gold.shape)
        sentences = []
        tokenize_sentences = []
        
        print ("Using Sentence Average Embedding...")
        
        # creating embedding for empty sentences (for padding)
        lhs = torch.empty((len(essays),max_sentences,768), dtype=torch.float)
        emb_for_padding = tokenizer.encode_plus("", add_special_tokens=True, truncation=True, padding="max_length", return_tensors="pt", max_length=10)
        tt = torch.tensor(emb_for_padding['input_ids'])
        output = model(tt)
        if (hstate=='second_last'):
            # getting the 2nd last layer
            lhs_for_padding = output.hidden_states[11]
        elif (hstate=='last4sum'):
            # getting the sum of last 4 hidden states
            lhs_for_padding = output.hidden_states[12] + output.hidden_states[11] + output.hidden_states[10] + output.hidden_states[9]
        else:
            print ("Invalid value provided for hstate")
            return

        lhs_for_padding_np = np.array(lhs_for_padding.detach().numpy())
        lhs_for_padding_mean = np.mean(lhs_for_padding_np,axis=1)
        lhs_avg_for_padding = torch.tensor(lhs_for_padding_mean[0])

        for j,essay in tqdm(enumerate(essays)):

            sentences = re.split('\. |\? |! ', essay)
            sen_length = len(sentences)
            lhs_sentence_avg = np.zeros((max_sentences,768), dtype=float)

            for i,s in enumerate(sentences):
                if (i>=max_sentences):
                    break
                tokenize_sentence = tokenizer.encode(sentences[i],add_special_tokens=True, max_length=512, truncation=True)
                tt = torch.tensor(tokenize_sentence)
                tts = tt.reshape(1,len(tt))
                output = model(tts)
                if (hstate=='second_last'):
                    # getting the 2nd last layer
                    lhs_sentence = output.hidden_states[11]
                elif (hstate=='last4sum'):
                    lhs_sentence = output.hidden_states[12] + output.hidden_states[11] + output.hidden_states[10] + output.hidden_states[9]
                else:
                    print ("Invalid value provided for hstate")
                    return
                
                lhs_sentence_np = np.array(lhs_sentence.detach().numpy())
                lhs_sentence_np_mean = np.mean(lhs_sentence_np,axis=1)
                lhs_sentence_avg[i] = lhs_sentence_np_mean[0]

            lhs[j] = torch.tensor(lhs_sentence_avg)

            if (sen_length < max_sentences):
                for i in range (sen_length, max_sentences):
                    lhs[j][i]= lhs_avg_for_padding
        sets.append((lhs,y_gold))
        cat_lhs = torch.cat((cat_lhs,lhs),0)

    torch.save(sets, sets_lhs_path)
    torch.save(cat_lhs, lhs_path)
    torch.save(cat_y_gold, y_path)
    
    print ("Returning lhs: Shape: ", lhs.shape)
    print ("Returning y_gold: Shape: ", y_gold.shape)
    print ("Returning lhs/golden sets: ",type(sets))
    
    return lhs, y_gold, sets
    

In [ ]:
def load_embeddings(directory, filename, emb_type, get_sets=True):
    lhs_path = directory + '/' + filename + '_lhs.pt'
    y_path = directory + '/' + filename + '_y.pt'
    sets_lhs_path = directory + '/' + filename + '_sets.pt'
    
    print ("Loading embeddings from directory: %s" %directory)
    print ("lhs file chosen: ", lhs_path)
    print ("y file chosen: ", y_path)
    print ("sets file chosen: ", sets_lhs_path)
    
    lhs = torch.load(lhs_path)
    y_gold = torch.load(y_path)
    
    print ("Loaded, Size of LHS embeddings: ", lhs.shape)
    print ("Loaded, Size of y Gold: ", y_gold.shape)
    print ("Loading lhs: Shape: ", lhs.shape)
    print ("Loading y_gold: Shape: ", y_gold.shape)
    if get_sets and emb_type=='test':
        sets = torch.load(sets_lhs_path)
        print("Loading lhs/golden sets: ",type(sets))
        return lhs, y_gold, sets
    else:
        return lhs, y_gold 

In [ ]:
def return_shuffle_prompt(p, prompts):
    essay_id = prompts.essay_set[prompts['prompt'] == p]
    essay_ids = list(range(1,9))
    essay_ids.remove(int(essay_id))
    ran_id = random.choice(essay_ids)
    return prompts.prompt[ran_id-1]

In [ ]:
def evaluate_model (model, lhs_test=None, y_test=None, sets=False, lhs_sets=None):
    if(sets):
        idx = 1
        for sset in lhs_sets:
            print("Set %d"%idx)
            set_lhs_test = sset[0]
            set_y_test = sset[1]
            y_pred = model.predict(set_lhs_test.numpy())
            tt1 = np.around(10*y_pred)
            tt2 = tt1.reshape(tt1.shape[0],)
            pred_values = tt2.astype(int)
            tt3 = np.array(10* set_y_test)
            gold_values = tt3.astype(int)
            # evaluate the model
            result = cohen_kappa_score(gold_values,pred_values,weights='quadratic')
            print("Kappa Score: {}".format(result))
            yy_p = y_pred.reshape(y_pred.shape[0],)
            yy_t = np.array(set_y_test)
            MSE = np.square(np.subtract(yy_t, yy_p)).mean()
            RMSE = math.sqrt(MSE)
            print ("MSE: ", MSE)
            print ("RMSE: %f\n" %RMSE)
            idx +=1
    if (len(lhs_test)!=0 and len(y_test)!=0): 
        print("Combined test set results: ")
        y_pred = model.predict(lhs_test.numpy())
        tt1 = np.around(10*y_pred)
        tt2 = tt1.reshape(tt1.shape[0],)
        pred_values = tt2.astype(int)
        tt3 = np.array(10* y_test)
        gold_values = tt3.astype(int)
        # evaluate the model
        result = cohen_kappa_score(gold_values,pred_values,weights='quadratic')
        print("Kappa Score: {}".format(result))
        yy_p = y_pred.reshape(y_pred.shape[0],)
        yy_t = np.array(y_test)
        MSE = np.square(np.subtract(yy_t, yy_p)).mean()
        RMSE = math.sqrt(MSE)
        print ("MSE: ", MSE)
        print ("RMSE: ", RMSE)

### Creating XPD Embeddings

In [ ]:
prompts = pd.read_csv('/home/anurag/Documents/agastya/AES/NEW/prompts-aes-kaggle-dataset.csv')
prompts = prompts.rename(columns={'essat_set':'essay_set'})

FileNotFoundError: ignored

In [ ]:
data = data.merge(prompts, left_on="essay_set", right_on='essay_set')
data['combined_essay'] = data['prompt'] + data['essay']

In [ ]:
# augmenting dataset (coherence)
coh_aug = data.sample(frac=0.25,random_state=42)
coh_aug.essay = coh_aug.essay.apply(lambda x: coherence_augment(x))
coh_aug = coh_aug.reset_index()
coh_aug = coh_aug.drop(columns=['index'])
coh_aug['normalized_score'] = coh_aug['normalized_score']* 0.75
xpd_dataset = data.append(coh_aug)

<ipython-input-17-4f1790e10932>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xpd_dataset = data.append(coh_aug)


In [ ]:
prel_aug = data.sample(frac=0.25, random_state=42)
prel_aug['prompt'] = prel_aug['prompt'].apply(lambda x: str(return_shuffle_prompt(x,prompts)))
prel_aug['normalized_score'] = prel_aug['normalized_score']* 0.75
# prel_aug['combined_essay'] = prel_aug['prompt'] + prel_aug['essay']

In [ ]:
both_aug = data.sample(frac=0.25,random_state=42)
both_aug['prompt'] = both_aug['prompt'].apply(lambda x: str(return_shuffle_prompt(x,prompts)))
both_aug.essay = both_aug.essay.apply(lambda x: coherence_augment(x))
both_aug['normalized_score'] = both_aug['normalized_score']* 0.5

In [ ]:
xpd_dataset = xpd_dataset.append(prel_aug)
xpd_dataset = xpd_dataset.append(both_aug)
xpd_dataset = xpd_dataset.reset_index()
xpd_dataset = xpd_dataset.drop(columns=['index'])

<ipython-input-20-fcfbba513b8a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xpd_dataset = xpd_dataset.append(prel_aug)
<ipython-input-20-fcfbba513b8a>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  xpd_dataset = xpd_dataset.append(both_aug)


In [ ]:
xpd_dataset = xpd_dataset.sample(frac=1, random_state=42)

In [ ]:
xpd_train, xpd_test = train_test_split(xpd_dataset, test_size=0.33)
print("XPD Train Shape: ",xpd_train.shape)
print("XPD Test Shape: ",xpd_test.shape)

XPD Train Shape:  (15214, 7)
XPD Test Shape:  (7494, 7)


In [ ]:
load_xpd_dataset = True
if load_xpd_dataset == False:
    xpd_train.to_csv("/home/anurag/Documents/agastya/AES/NEW/xpd_train_new.csv")
    xpd_test.to_csv("/home/anurag/Documents/agastya/AES/NEW/xpd_test_new.csv")
    xpd_dataset.to_csv("/home/anurag/Documents/agastya/AES/NEW/xpd_dataset_new.csv")
else:
    xpd_train = pd.read_csv("/home/anurag/Documents/agastya/AES/NEW/xpd_train_new.csv")
    xpd_test = pd.read_csv("/home/anurag/Documents/agastya/AES/NEW/xpd_test_new.csv")
    xpd_dataset = pd.read_csv("/home/anurag/Documents/agastya/AES/NEW/xpd_dataset_new.csv")

In [ ]:
experiment_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW/'

#### XPD Train Embeddings

In [ ]:
load_xpd_embeddings = True

if not load_xpd_embeddings:
    lhs_train_xpd, y_train_xpd, lhs_sets_xpd = prepare_embeddings(xpd_train, file_name='xpd_train', file_path=experiment_path)
    lhs_test_xpd, y_test_xpd, lhs_test_sets_xpd = prepare_embeddings(xpd_test, file_name='xpd_test', file_path=experiment_path)
else:
    lhs_train_xpd, y_train_xpd, lhs_sets_train_xpd = load_embeddings(directory=experiment_path, filename='xpd_train')
    lhs_test_xpd, y_test_xpd, lhs_sets_test_xpd = load_embeddings(directory=experiment_path, filename='xpd_test')

Loading embeddings from directory: /home/anurag/Documents/agastya/AES/NEW/
lhs file chosen:  /home/anurag/Documents/agastya/AES/NEW//xpd_train_lhs.pt
y file chosen:  /home/anurag/Documents/agastya/AES/NEW//xpd_train_y.pt
sets file chosen:  /home/anurag/Documents/agastya/AES/NEW//xpd_train_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([15214, 128, 768])
Loaded, Size of y Gold:  (15214,)
Loading lhs: Shape:  torch.Size([15214, 128, 768])
Loading y_gold: Shape:  (15214,)
Loading lhs/golden sets:  <class 'list'>


### BPD Dataset

In [ ]:
bpd_dataset = data
bpd_train, bpd_test = train_test_split(bpd_dataset, test_size=0.33)

### Semantic Model Embeddings

In [ ]:
#lhs_train_bpd, y_train_bpd, lhs_sets_train_bpd = prepare_embeddings(bpd_train, file_name='bpd_train', file_path=experiment_path)
prepare_embeddings(bpd_train, file_name='bpd_train', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /home/anurag/Documents/agastya/AES/NEW/
Save File Name:  bpd_train
Dataframe provided, Size:  (8693, 7)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-6ffa86edcb03>:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a 

Creating embeddings for set: 1
Shape of the set:  (1204, 7)
Concatenated y_gold shape:  (1204,)
Using Sentence Average Embedding...


1204it [18:16,  1.10it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 2
Shape of the set:  (1206, 7)
Concatenated y_gold shape:  (2410,)
Using Sentence Average Embedding...


418it [05:56,  1.14it/s]

In [ ]:
#lhs_test_bpd, y_test_bpd, lhs_sets_test_bpd = prepare_embeddings(bpd_test, file_name='bpd_test', file_path=experiment_path)
prepare_embeddings(bpd_test, file_name='bpd_test', file_path=experiment_path)

### Coherence Model Embeddings

#### Creating Augmented Dataset

In [ ]:
print ("Original Train Data Shape", bpd_train.shape)
print ("Original Test Data Shape", bpd_test.shape)

samp_bpd_train = bpd_train.sample(frac=0.33, random_state=42)
samp_bpd_train['essay'] = samp_bpd_train['essay'].apply(lambda x: coherence_augment(x))
samp_bpd_train['normalized_score'] = 0
aug_bpd_train_coh = bpd_train.append(samp_bpd_train)
aug_bpd_train_coh = aug_bpd_train_coh.reset_index()
aug_bpd_train_coh = aug_bpd_train_coh.drop(columns=['domain1_score','index'])
aug_bpd_train_coh = aug_bpd_train_coh.sample(frac=1, random_state=42)
print ("Augmented Train Data Shape", aug_bpd_train_coh.shape)

samp_bpd_test = bpd_test.sample(frac=0.33, random_state=42)
samp_bpd_test['essay'] = samp_bpd_test['essay'].apply(lambda x: coherence_augment(x))
samp_bpd_test['normalized_score'] = 0
aug_bpd_test_coh = bpd_test.append(samp_bpd_test)
aug_bpd_test_coh = aug_bpd_test_coh.reset_index()
aug_bpd_test_coh = aug_bpd_test_coh.drop(columns=['domain1_score','index'])
aug_bpd_test_coh = aug_bpd_test_coh.sample(frac=1, random_state=42)
print ("Augmented Test Data Shape", aug_bpd_test_coh.shape)

Original Train Data Shape (8693, 5)
Original Test Data Shape (4283, 5)
Augmented Train Data Shape (11562, 4)
Augmented Test Data Shape (5696, 4)


In [ ]:
#lhs_train_bpd_coh, y_train_bpd_coh, lhs_sets_train_bpd_coh = prepare_embeddings(aug_bpd_train_coh, file_name='bpd_train_coh', file_path=experiment_path)
prepare_embeddings(aug_bpd_train_coh, file_name='bpd_train_coh', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /home/anurag/Documents/agastya/AES/NEW/
Save File Name:  bpd_train_coh
Dataframe provided, Size:  (11562, 6)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-6ffa86edcb03>:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a 

Creating embeddings for set: 1
Shape of the set:  (1579, 6)
Concatenated y_gold shape:  (1579,)
Using Sentence Average Embedding...


1579it [23:59,  1.10it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 2
Shape of the set:  (1613, 6)
Concatenated y_gold shape:  (3192,)
Using Sentence Average Embedding...


...

1384it [10:35,  2.18it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 8
Shape of the set:  (641, 6)
Concatenated y_gold shape:  (11562,)
Using Sentence Average Embedding...


641it [14:40,  1.37s/it]


Returning lhs: Shape:  torch.Size([641, 128, 768])
Returning y_gold: Shape:  (641,)
Returning lhs/golden sets:  <class 'list'>


(tensor([[[-5.2224e-01,  1.5750e+00, -5.1078e-01,  ..., -2.4720e-02,
            6.1356e-01, -1.0603e-01],
          [-6.9733e-01,  8.6583e-01,  8.4768e-01,  ...,  1.1589e-01,
            6.8404e-02,  4.6224e-01],
          [-3.1369e-01,  1.1582e+00,  5.5510e-01,  ..., -1.7101e+00,
            1.7268e+00, -8.2411e-01],
          ...,
          [-4.8699e-01, -2.6756e+00,  2.7683e+00,  ..., -1.7928e+00,
            1.6839e+00, -2.2029e+00],
          [-4.8699e-01, -2.6756e+00,  2.7683e+00,  ..., -1.7928e+00,
            1.6839e+00, -2.2029e+00],
          [-4.8699e-01, -2.6756e+00,  2.7683e+00,  ..., -1.7928e+00,
            1.6839e+00, -2.2029e+00]],
 
         [[-3.2479e-01,  1.3500e+00, -3.5440e-01,  ..., -1.5345e+00,
           -9.8187e-01,  1.3712e+00],
          [-2.0102e+00,  1.0573e+00, -2.2252e-01,  ..., -2.2342e+00,
           -1.1015e+00,  1.9760e-02],
          [-1.4952e+00, -6.0220e-01,  6.8640e-01,  ..., -8.6751e-01,
            2.3908e-01,  5.7091e-01],
          ...,
    

In [ ]:
#lhs_test_bpd_coh, y_test_bpd_coh, lhs_sets_test_bpd_coh = prepare_embeddings(aug_bpd_test_coh, file_name='bpd_test_coh', file_path=experiment_path)
prepare_embeddings(aug_bpd_test_coh, file_name='bpd_test_coh', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /home/anurag/Documents/agastya/AES/NEW/
Save File Name:  bpd_test_coh
Dataframe provided, Size:  (5696, 6)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-6ffa86edcb03>:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a 

Creating embeddings for set: 1
Shape of the set:  (779, 6)
Concatenated y_gold shape:  (779,)
Using Sentence Average Embedding...


779it [12:03,  1.08it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 2
Shape of the set:  (780, 6)
Concatenated y_gold shape:  (1559,)
Using Sentence Average Embedding...


...

701it [05:26,  2.15it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 8
Shape of the set:  (331, 6)
Concatenated y_gold shape:  (5696,)
Using Sentence Average Embedding...


331it [07:58,  1.45s/it]


Returning lhs: Shape:  torch.Size([331, 128, 768])
Returning y_gold: Shape:  (331,)
Returning lhs/golden sets:  <class 'list'>


(tensor([[[-1.7399,  0.6974,  0.6018,  ...,  0.9116,  0.5870,  0.6531],
          [ 0.5872,  1.2479,  0.8745,  ..., -0.9373,  0.1530, -0.8729],
          [-0.8316,  0.4581,  1.2931,  ..., -0.7903,  0.7551, -0.4595],
          ...,
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029]],
 
         [[-1.7661, -0.9421,  0.9515,  ..., -2.2517,  1.4425, -1.6623],
          [-0.3761, -2.7718,  1.0282,  ..., -1.6878, -0.2692, -0.5139],
          [ 0.5481, -0.0110,  2.7119,  ..., -1.9940,  2.3146, -1.4008],
          ...,
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029]],
 
         [[ 1.3612, -0.9988, -0.7675,  ..., -1.5716,  0.3562,  0.0645],
          [-0.7646,  0.1073,

### Prel Model Embeddings

In [ ]:
bpd_train_prel = bpd_train

In [ ]:
bpd_test_prel = bpd_test

In [ ]:
bpd_train.head()

,essay_id,essay_set,essay,domain1_score,normalized_score
8830,13578,5,A wonderful mood was created by Narciso Rodrig...,3,0.750000
5672,9227,4,The author uses this sentence to conclude the ...,2,0.666667
6771,10330,4,Saeng will take the test in the spring when it...,1,0.333333
7783,12531,5,"In the memoir, , from Home: The Blueprints of ...",4,1.000000
9409,15359,6,There were a few obstacles builders of the Emp...,2,0.500000


In [ ]:
prompts = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AES/prompts-aes-kaggle-dataset.csv')
prompts = prompts.rename(columns={'essat_set':'essay_set'})

In [ ]:
bpd_train_prel = bpd_train.merge(prompts,on='essay_set')
bpd_test_prel  = bpd_test.merge(prompts,on='essay_set')

In [ ]:
bpd_train_prel['combined_essay'] = bpd_train_prel['prompt'] + bpd_train_prel['essay']
bpd_test_prel['combined_essay']  = bpd_test_prel['prompt'] + bpd_train_prel['essay']

In [ ]:
samp_prompt_train = bpd_train_prel.sample(frac=0.33, random_state=42)
samp_prompt_train['prompt'] = samp_prompt_train['prompt'].apply(lambda x: str(return_shuffle_prompt(x,prompts)))
samp_prompt_train['combined_essay'] = samp_prompt_train['prompt'] + samp_prompt_train['essay']
aug_bpd_train_prel = bpd_train_prel.append(samp_prompt_train)
aug_bpd_train_prel = aug_bpd_train_prel.reset_index()
aug_bpd_train_prel = aug_bpd_train_prel.drop(columns=['index'])
aug_bpd_train_prel = aug_bpd_train_prel.sample(frac=1, random_state=42)

In [ ]:
samp_prompt_test = bpd_test_prel.sample(frac=0.33, random_state=42)
samp_prompt_test['prompt'] = samp_prompt_test['prompt'].apply(lambda x: str(return_shuffle_prompt(x,prompts)))
samp_prompt_test['combined_essay'] = samp_prompt_test['prompt'] + samp_prompt_test['essay']
aug_bpd_test_prel = bpd_test_prel.append(samp_prompt_test)
aug_bpd_test_prel = aug_bpd_test_prel.reset_index()
aug_bpd_test_prel = aug_bpd_test_prel.drop(columns=['index'])
aug_bpd_test_prel = aug_bpd_test_prel.sample(frac=1, random_state=42)

In [ ]:
lhs_train_bpd_prel, y_train_bpd_prel, lhs_sets_train_bpd_prel = prepare_embeddings(aug_bpd_train_prel, file_name='bpd_train_prel', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /home/anurag/Documents/agastya/AES/NEW/
Save File Name:  bpd_train_prel
Dataframe provided, Size:  (11562, 7)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-6ffa86edcb03>:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a 

Creating embeddings for set: 1
Shape of the set:  (1527, 7)
Concatenated y_gold shape:  (1527,)
Using Sentence Average Embedding...


1527it [23:20,  1.09it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 2
Shape of the set:  (1608, 7)
Concatenated y_gold shape:  (3135,)
Using Sentence Average Embedding...


...

Creating embeddings for set: 7
Shape of the set:  (1398, 7)
Concatenated y_gold shape:  (10891,)
Using Sentence Average Embedding...


1398it [10:24,  2.24it/s]


Creating embeddings for set: 8
Shape of the set:  (671, 7)
Concatenated y_gold shape:  (11562,)
Using Sentence Average Embedding...


<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])
671it [15:26,  1.38s/it]


Returning lhs: Shape:  torch.Size([671, 128, 768])
Returning y_gold: Shape:  (671,)
Returning lhs/golden sets:  <class 'list'>


In [ ]:
lhs_test_bpd_prel, y_test_bpd_prel, lhs_sets_test_bpd_prel = prepare_embeddings(aug_bpd_test_prel, file_name='bpd_test_prel', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /home/anurag/Documents/agastya/AES/NEW/
Save File Name:  bpd_test_prel
Dataframe provided, Size:  (5696, 7)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-12-6ffa86edcb03>:56: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a 

Creating embeddings for set: 1
Shape of the set:  (848, 7)
Concatenated y_gold shape:  (848,)
Using Sentence Average Embedding...


848it [12:52,  1.10it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 2
Shape of the set:  (785, 7)
Concatenated y_gold shape:  (1633,)
Using Sentence Average Embedding...


...

Creating embeddings for set: 7
Shape of the set:  (663, 7)
Concatenated y_gold shape:  (5419,)
Using Sentence Average Embedding...


663it [05:27,  2.02it/s]
<ipython-input-12-6ffa86edcb03>:65: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cat_y_gold = cat_y_gold.append(y_gold)
<ipython-input-12-6ffa86edcb03>:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tt = torch.tensor(emb_for_padding['input_ids'])


Creating embeddings for set: 8
Shape of the set:  (277, 7)
Concatenated y_gold shape:  (5696,)
Using Sentence Average Embedding...


277it [06:26,  1.40s/it]


Returning lhs: Shape:  torch.Size([277, 128, 768])
Returning y_gold: Shape:  (277,)
Returning lhs/golden sets:  <class 'list'>


### Coherence Model

#### Loading Embeddings

In [ ]:

experiment_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW/'
lhs_train_coh, y_train_coh = load_embeddings(directory=experiment_path, emb_type='train', filename='bpd_train_coh')

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_coh_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_coh_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_coh_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([11562, 128, 768])
Loaded, Size of y Gold:  (11562,)
Loading lhs: Shape:  torch.Size([11562, 128, 768])
Loading y_gold: Shape:  (11562,)


In [ ]:
experiment_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW/'
bpd_coh_model_save_path = experiment_path + '/bpd_coh_model.pt'

In [ ]:
load_bpd_trained_model_coh = True
if (load_bpd_trained_model_coh == True):
    lstm_model_coh_nsp = load_model(bpd_coh_model_save_path)
else:
    lstm_model_coh_nsp = get_model(Hidden_dim1=1028, Hidden_dim2=512, return_sequences = True, dropout_dense=0.5, dropout_lstm=0.4, 
                             recurrent_dropout=0.4, sen_size=max_sentences, input_size=768, activation='sigmoid', 
                             opt_engine='adam', loss_fn='mse')
    lstm_model_coh_nsp.fit(lhs_train_coh.numpy(), y_train_coh, batch_size=64, epochs=100)
    lstm_model_coh_nsp.save(bpd_coh_model_save_path)

Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


In [ ]:
lhs_test_coh, y_test_coh, lhs_test_sets_coh = load_embeddings(directory=experiment_path, emb_type='test', filename='bpd_test_coh')
evaluate_model (lstm_model_coh_nsp, lhs_test=lhs_test_coh, y_test=y_test_coh, sets=True, lhs_sets=lhs_test_sets_coh)

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_coh_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_coh_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_coh_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([5696, 128, 768])
Loaded, Size of y Gold:  (5696,)
Loading lhs: Shape:  torch.Size([5696, 128, 768])
Loading y_gold: Shape:  (5696,)
Loading lhs/golden sets:  <class 'list'>
Set 1
Kappa Score: 0.8730150969295594
MSE:  0.021193840306537984
RMSE: 0.145581

Set 2
Kappa Score: 0.6945774718148259
MSE:  0.030563502324134156
RMSE: 0.174824

Set 3
Kappa Score: 0.736110918921176
MSE:  0.05380106183035603
RMSE: 0.231951

Set 4
Kappa Score: 0.7414732861599784
MSE:  0.05033033128662709
RMSE: 0.224344

Set 5
Kappa Score: 0.8235985979704431
MSE:  0.03465241615205221
RMSE: 0.186152

Set 6
Kappa Score: 0.8316937741560204
MSE:

### Prompt relevance model

#### Loading Embeddings

In [ ]:
del lhs_test_coh, y_test_coh, lhs_test_sets_coh

In [ ]:
experiment_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW/'
lhs_train_prel, y_train_prel = load_embeddings(directory=experiment_path, emb_type='train', filename='bpd_train_prel')

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_prel_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_prel_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_prel_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([11562, 128, 768])
Loaded, Size of y Gold:  (11562,)
Loading lhs: Shape:  torch.Size([11562, 128, 768])
Loading y_gold: Shape:  (11562,)


In [ ]:
prel_model_save_path = experiment_path + '/bpd_prel_model.pt'

In [ ]:
load_trained_model_prel = True
if (load_trained_model_prel == True):
    lstm_model_prel = load_model(prel_model_save_path)
else:
    lstm_model_prel = get_model(Hidden_dim1=1028, Hidden_dim2=512, return_sequences = True, dropout_dense=0.5, dropout_lstm=0.4, 
                                recurrent_dropout=0.4, sen_size=max_sentences, input_size=768, activation='sigmoid', 
                                opt_engine='adam', loss_fn='mse')
    lstm_model_prel.fit(lhs_train_prel.numpy(), y_train_prel, batch_size=96, epochs=100)
    lstm_model_prel.save(prel_model_save_path)

Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


In [ ]:
lhs_test_prel, y_test_prel, lhs_test_sets_prel = load_embeddings(directory=experiment_path, emb_type='test', filename='bpd_test_prel')
evaluate_model (lstm_model_prel, lhs_test=lhs_test_prel, y_test=y_test_prel, sets=True, lhs_sets=lhs_test_sets_prel)

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_prel_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_prel_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_prel_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([5696, 128, 768])
Loaded, Size of y Gold:  (5696,)
Loading lhs: Shape:  torch.Size([5696, 128, 768])
Loading y_gold: Shape:  (5696,)
Loading lhs/golden sets:  <class 'list'>
Set 1
Kappa Score: 0.7270551385429662
MSE:  0.009770465629389206
RMSE: 0.098846

Set 2
Kappa Score: 0.6511267168104515
MSE:  0.012828977382884335
RMSE: 0.113265

Set 3
Kappa Score: 0.6385167145269592
MSE:  0.043796389855832965
RMSE: 0.209276

Set 4
Kappa Score: 0.7883295118958125
MSE:  0.032359156119945255
RMSE: 0.179887

Set 5
Kappa Score: 0.7893283768687873
MSE:  0.019971468130892785
RMSE: 0.141320

Set 6
Kappa Score: 0.72545733994731

### Semantic Model

In [ ]:
experiment_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW/'

In [ ]:
lhs_train_sem, y_train_sem = load_embeddings(directory=experiment_path, emb_type='train', filename='bpd_train')

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([8693, 128, 768])
Loaded, Size of y Gold:  (8693,)
Loading lhs: Shape:  torch.Size([8693, 128, 768])
Loading y_gold: Shape:  (8693,)


In [ ]:
bpd_sem_model_save_path = experiment_path + '/bpd_sem_model-nnn2.pt'

In [ ]:
load_bpd_trained_model_sem = False
if (load_bpd_trained_model_sem == True):
    lstm_model_sem = load_model(bpd_sem_model_save_path)
else:
    lstm_model_sem = get_model(Hidden_dim1=1028, Hidden_dim2=512, return_sequences = True, dropout_dense=0.5, dropout_lstm=0.4, 
                             recurrent_dropout=0.4, sen_size=max_sentences, input_size=768, activation='sigmoid', 
                             opt_engine='adam', loss_fn='mse')
    lstm_model_sem.fit(lhs_train_sem.numpy(), y_train_sem, batch_size=64, epochs=100)
    lstm_model_sem.save(bpd_sem_model_save_path)

Layer lstm_2 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Layer lstm_3 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 768, 1028)         4757584   
                                                                 
 lstm_3 (LSTM)               (None, 512)               3155968   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 7,914,065
Trainable params: 7,914,065
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100


ValueError: ignored

In [ ]:
bpd_sem_model_save_path = '/content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_sem_model-nnn2.pt'
lstm_model_sem = load_model(bpd_sem_model_save_path)

Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


In [ ]:
lhs_test_sem, y_test_sem, lhs_test_sets_sem = load_embeddings(directory=experiment_path, emb_type='test', filename='bpd_test')
evaluate_model (lstm_model_sem, lhs_test=lhs_test_sem, y_test=y_test_sem, sets=True, lhs_sets=lhs_test_sets_sem)

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_test_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([4283, 128, 768])
Loaded, Size of y Gold:  (4283,)
Loading lhs: Shape:  torch.Size([4283, 128, 768])
Loading y_gold: Shape:  (4283,)
Loading lhs/golden sets:  <class 'list'>
Set 1
Kappa Score: 0.7098951237286525
MSE:  0.011164804296248639
RMSE: 0.105664

Set 2
Kappa Score: 0.5968137634789539
MSE:  0.018895198980263508
RMSE: 0.137460

Set 3
Kappa Score: 0.6817170453580075
MSE:  0.04193638277351106
RMSE: 0.204784

Set 4
Kappa Score: 0.8128130037805714
MSE:  0.03195000933118899
RMSE: 0.178746

Set 5
Kappa Score: 0.7986907952025376
MSE:  0.024208315965290307
RMSE: 0.155590

Set 6
Kappa Score: 0.7954650537255166
MSE:  0.020443

# Load Augmented DataSet to build 2nd Stage Model (XGBoost or RF)

- Augmentation of Data as:
   1. Original vectors with Final (Domain) score as is
   2. Augment sentence jumbled dataset and degrade overall score on 25%
   3. Augment prompt jumbled dataset and degrade overall score by 25%
   4. Augment both sentence + prompt jumbled datased and degrade overall score by 50%

# Create Semantic Embeddings: second_last_state

In [ ]:
prepare_embeddings(bpd_train, file_name='bpd_train_sec_last_as', file_path=experiment_path, hstate="second_last")

Preparing Embeddings...
hState:  second_last
Save File Directory:  /content/drive/MyDrive/Colab Notebooks/AES/NEW/
Save File Name:  bpd_train_sec_last_as
Dataframe provided, Size:  (8693, 5)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

...

1046it [12:51,  1.36it/s]


Creating embeddings for set: 8
Shape of the set:  (480, 5)
Concatenated y_gold shape:  (8693,)
Using Sentence Average Embedding...


480it [17:22,  2.17s/it]


Returning lhs: Shape:  torch.Size([480, 128, 768])
Returning y_gold: Shape:  (480,)
Returning lhs/golden sets:  <class 'list'>


(tensor([[[ 3.7900e-01,  2.6653e-01,  1.6800e-01,  ..., -3.4415e-01,
           -2.8845e-01,  2.1745e-02],
          [-9.2748e-01, -5.6732e-01,  1.5034e-01,  ..., -6.7746e-01,
            3.3901e-02,  3.6634e-01],
          [ 2.4279e-01,  1.9278e-01, -1.5932e-01,  ..., -1.3875e-01,
           -2.0994e-02, -2.9803e-01],
          ...,
          [-1.2540e-01, -7.7911e-01,  7.6463e-01,  ..., -5.5709e-01,
            4.8292e-01, -6.2657e-01],
          [-1.2540e-01, -7.7911e-01,  7.6463e-01,  ..., -5.5709e-01,
            4.8292e-01, -6.2657e-01],
          [-1.2540e-01, -7.7911e-01,  7.6463e-01,  ..., -5.5709e-01,
            4.8292e-01, -6.2657e-01]],
 
         [[ 4.2284e-01,  1.8719e-01,  3.0013e-01,  ..., -3.3656e-01,
            1.7649e-01, -4.2838e-02],
          [ 8.1931e-02, -2.2317e-02, -3.3620e-02,  ..., -1.2542e-01,
           -2.6899e-01, -1.4158e-01],
          [-2.0823e-01,  7.1715e-02, -6.9515e-03,  ..., -4.9370e-01,
           -2.7763e-02, -1.1357e-01],
          ...,
    

In [ ]:
prepare_embeddings(bpd_test, file_name='bpd_test_sec_last_as', file_path=experiment_path, hstate="second_last")

Preparing Embeddings...
hState:  second_last
Save File Directory:  /content/drive/MyDrive/Colab Notebooks/AES/NEW/
Save File Name:  bpd_test_sec_last_as
Dataframe provided, Size:  (4283, 5)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

...

523it [05:49,  1.49it/s]


Creating embeddings for set: 8
Shape of the set:  (241, 5)
Concatenated y_gold shape:  (4283,)
Using Sentence Average Embedding...


241it [09:12,  2.29s/it]


Returning lhs: Shape:  torch.Size([241, 128, 768])
Returning y_gold: Shape:  (241,)
Returning lhs/golden sets:  <class 'list'>


(tensor([[[ 0.3231,  0.0687,  0.3769,  ..., -0.3777,  0.4022,  0.1629],
          [-0.0015,  0.0305, -0.4217,  ...,  0.1287, -0.0833,  0.0446],
          [ 0.1505,  0.0576,  0.1022,  ..., -0.3716, -0.3493,  0.1465],
          ...,
          [-0.1254, -0.7791,  0.7646,  ..., -0.5571,  0.4829, -0.6266],
          [-0.1254, -0.7791,  0.7646,  ..., -0.5571,  0.4829, -0.6266],
          [-0.1254, -0.7791,  0.7646,  ..., -0.5571,  0.4829, -0.6266]],
 
         [[-0.2643,  0.0751, -0.2231,  ..., -0.3418, -0.1049, -0.1319],
          [-0.1336, -0.1367, -0.0987,  ...,  0.2325, -0.0874,  0.0089],
          [-0.1592, -0.2682,  0.0064,  ..., -0.7416,  0.1171,  0.3348],
          ...,
          [-0.1254, -0.7791,  0.7646,  ..., -0.5571,  0.4829, -0.6266],
          [-0.1254, -0.7791,  0.7646,  ..., -0.5571,  0.4829, -0.6266],
          [-0.1254, -0.7791,  0.7646,  ..., -0.5571,  0.4829, -0.6266]],
 
         [[ 0.8620,  0.7311, -0.1086,  ...,  0.0233,  0.1975,  0.2366],
          [-0.1583, -0.3320,

# Build Semantic Model using last second last state in embeddings

In [ ]:
lhs_train_sem_sl, y_train_sem_sl = load_embeddings(directory=experiment_path, emb_type='train', filename='bpd_train_sec_last_as')

Loading embeddings from directory: /content/drive/MyDrive/Colab Notebooks/AES/NEW/
lhs file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_sec_last_as_lhs.pt
y file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_sec_last_as_y.pt
sets file chosen:  /content/drive/MyDrive/Colab Notebooks/AES/NEW//bpd_train_sec_last_as_sets.pt
Loaded, Size of LHS embeddings:  torch.Size([8693, 128, 768])
Loaded, Size of y Gold:  (8693,)
Loading lhs: Shape:  torch.Size([8693, 128, 768])
Loading y_gold: Shape:  (8693,)


In [ ]:
bpd_sem_model_sl_save_path = experiment_path + '/bpd_sem_model_sl.pt'

In [ ]:
load_bpd_trained_model_sem_sl = False
if (load_bpd_trained_model_sem_sl == True):
    lstm_model_sem_sl = load_model(bpd_sem_model_sl_save_path)
else:
    lstm_model_sem_sl = get_model(Hidden_dim1=1028, Hidden_dim2=512, return_sequences = True, dropout_dense=0.5, dropout_lstm=0.4, 
                             recurrent_dropout=0.4, sen_size=max_sentences, input_size=768, activation='linear', 
                             opt_engine='RMSProp', loss_fn='mse')
    lstm_model_sem_sl.fit(lhs_train_sem_sl.numpy(), y_train_sem_sl, batch_size=128, epochs=100)
    lstm_model_sem_sl.save(bpd_sem_model_sl_save_path)

Layer lstm will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Layer lstm_1 will not use cuDNN kernels since it doesn't meet the criteria. It will use a generic GPU kernel as fallback when running on GPU.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128, 1028)         7389264   
                                                                 
 lstm_1 (LSTM)               (None, 512)               3155968   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 10,545,745
Trainable params: 10,545,745
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
68/68 [==============================] - 54s 700ms/step - loss: 2.3670 - mae: 0.5021
Epoch 2/100
68/68 [==============================] - 48s 703ms/step - loss: 0.0

In [ ]:
lhs_test_sem_sl, y_test_sem_sl, lhs_test_sets_sem_sl = load_embeddings(directory=experiment_path, emb_type='test', filename='bpd_test_sec_last_as')
evaluate_model (lstm_model_sem_sl, lhs_test=lhs_test_sem_sl, y_test=y_test_sem_sl, sets=True, lhs_sets=lhs_test_sets_sem_sl)

# Create Coherence Embeddings: second_last_state

In [ ]:
prepare_embeddings(aug_bpd_train_coh, file_name='bpd_train_coh_sec_last_as', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /content/drive/MyDrive/Colab Notebooks/AES/NEW/
Save File Name:  bpd_train_coh_sec_last_as
Dataframe provided, Size:  (11562, 4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float6

Creating embeddings for set: 1
Shape of the set:  (1606, 4)
Concatenated y_gold shape:  (1606,)
Using Sentence Average Embedding...


1606it [38:55,  1.45s/it]


Creating embeddings for set: 2
Shape of the set:  (1627, 4)
Concatenated y_gold shape:  (3233,)
Using Sentence Average Embedding...


...

1390it [16:00,  1.45it/s]


Creating embeddings for set: 8
Shape of the set:  (632, 4)
Concatenated y_gold shape:  (11562,)
Using Sentence Average Embedding...


632it [22:07,  2.10s/it]


Returning lhs: Shape:  torch.Size([632, 128, 768])
Returning y_gold: Shape:  (632,)
Returning lhs/golden sets:  <class 'list'>


(tensor([[[ 0.9584,  0.6165, -0.0166,  ..., -0.8199,  0.8487, -0.9130],
          [-0.1596,  1.7017,  1.3090,  ..., -1.9324,  2.1760, -1.4029],
          [-0.4007, -1.7083,  1.4183,  ..., -3.8129,  2.4229, -2.4116],
          ...,
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029]],
 
         [[-0.4665, -1.0047,  0.8514,  ..., -1.8573,  0.3775, -1.1740],
          [ 0.5660, -0.4135,  0.3295,  ..., -1.0381, -0.5786,  0.6653],
          [-0.7882, -1.8942,  1.0869,  ..., -2.0551,  0.3349, -0.3392],
          ...,
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029]],
 
         [[ 0.2616,  0.1892,  0.5110,  ..., -1.6497,  1.1181,  0.1484],
          [ 1.3460,  1.1776,

In [ ]:
prepare_embeddings(aug_bpd_test_coh, file_name='bpd_test_coh_sec_last_as', file_path=experiment_path)

Preparing Embeddings...
hState:  last4sum
Save File Directory:  /content/drive/MyDrive/Colab Notebooks/AES/NEW/
Save File Name:  bpd_test_coh_sec_last_as
Dataframe provided, Size:  (5696, 4)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float6

Creating embeddings for set: 1
Shape of the set:  (797, 4)
Concatenated y_gold shape:  (797,)
Using Sentence Average Embedding...


797it [19:27,  1.46s/it]


Creating embeddings for set: 2
Shape of the set:  (750, 4)
Concatenated y_gold shape:  (1547,)
Using Sentence Average Embedding...


...

706it [08:15,  1.42it/s]


Creating embeddings for set: 8
Shape of the set:  (312, 4)
Concatenated y_gold shape:  (5696,)
Using Sentence Average Embedding...


312it [12:21,  2.38s/it]


Returning lhs: Shape:  torch.Size([312, 128, 768])
Returning y_gold: Shape:  (312,)
Returning lhs/golden sets:  <class 'list'>


(tensor([[[-0.7547, -0.1124,  0.5894,  ..., -2.6914,  1.3229, -1.2469],
          [ 0.9221,  0.9103, -1.5369,  ...,  0.0873,  0.6719, -1.3936],
          [ 1.4885,  1.3010, -1.1737,  ..., -0.4506,  0.7839, -0.4907],
          ...,
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029]],
 
         [[-0.5681,  1.5557,  0.8234,  ...,  0.1220,  0.9628,  1.2944],
          [-1.4854, -1.3570,  1.6878,  ..., -0.4700,  1.7588,  0.6410],
          [ 0.3912,  0.4102, -0.3499,  ..., -1.2909,  0.3544, -0.6475],
          ...,
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029],
          [-0.4870, -2.6756,  2.7683,  ..., -1.7928,  1.6839, -2.2029]],
 
         [[-0.5166,  1.2120,  0.1357,  ..., -0.4121, -1.1007, -0.4758],
          [ 0.5883,  2.3621,

#Build Coherence Model using embeddings with second last stage

In [ ]:
lhs_train_coh_sl, y_train_coh_sl = load_embeddings(directory=experiment_path, emb_type='train', filename='bpd_train_coh_sec_last_as')

In [ ]:
bpd_coh_model_sl_save_path = experiment_path + '/bpd_coh_model_sl.pt'

In [ ]:
load_bpd_trained_model_coh_sl = False
if (load_bpd_trained_model_coh_sl == True):
    lstm_model_coh_sl = load_model(bpd_coh_model_sl_save_path)
else:
    lstm_model_coh_sl = get_model(Hidden_dim1=1028, Hidden_dim2=512, return_sequences = True, dropout_dense=0.5, dropout_lstm=0.4, 
                             recurrent_dropout=0.4, sen_size=max_sentences, input_size=768, activation='sigmoid', 
                             opt_engine='adam', loss_fn='mse')
    lstm_model_coh_sl.fit(lhs_train_coh_sl.numpy(), y_train_coh_sl, batch_size=64, epochs=100)
    lstm_model_coh_sl.save(bpd_coh_model_sl_save_path)

In [ ]:
lhs_test_coh_sl, y_test_coh_sl, lhs_test_sets_coh_sl = load_embeddings(directory=experiment_path, emb_type='test', filename='bpd_test_coh_sec_last_as')
evaluate_model (lstm_model_coh_sl, lhs_test=lhs_test_coh_sl, y_test=y_test_coh_sl, sets=True, lhs_sets=lhs_test_sets_coh_sl)

# Create Prompt Relevance Embeddings: second_last_state

In [ ]:
prepare_embeddings(aug_bpd_train_prel, file_name='bpd_train_prel_sec_last_as', file_path=experiment_path)

In [ ]:
prepare_embeddings(aug_bpd_test_prel, file_name='bpd_test_prel_sec_last_as', file_path=experiment_path)

## NEW ANURAG

In [ ]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.2 MB/s 
     |████████████████████████████████| 4.0 MB 23.4 MB/s 
     |████████████████████████████████| 1.2 MB 78.7 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 67.8 MB/s 
     |████████████████████████████████| 895 kB 65.9 MB/s 
     |████████████████████████████████| 596 kB 85.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=693f33c1f17d11623aae1be4b5cc661cd06c066778372463fdc88cd052e7765f
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import nltk
import torch
import re
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import pickle
from tqdm import tqdm
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
max_num_sentences = 128

In [ ]:
def sentence_tokenize(documents):
    """Clause function, which divides a whole paragraph of text into sentences

    Args:
        documents: document to be segmented, string type

    Returns: list of sentences

    """
    # Check
    locations = [-1]
    locations.extend([item.start() for item in re.finditer(r'[\.\?\!](?=[^ \W\d])', documents)])
    locations.append(len(documents))
    sentences = [documents[locations[i] + 1:locations[i + 1] + 1] for i in range(len(locations) - 1)]
    pre_split_documents = " ".join(sentences)

    sentences = nltk.sent_tokenize(pre_split_documents)
    return sentences

In [ ]:
#Loading the dataset
dataset_path = "/content/drive/MyDrive/Colab Notebooks/AES/training_set_rel3.tsv"
data = pd.read_csv(dataset_path, sep="\t", encoding="ISO-8859-1")
min_scores = [2, 1, 0, 0, 0, 0, 0, 0]
max_scores = [12, 6, 3, 3, 4, 4, 30, 60]
data.dropna(axis=1, inplace=True)
data.drop(columns=["rater1_domain1", "rater2_domain1"], inplace=True)
data['normalized_score'] = data.apply(lambda x: float(normalize_value(x['domain1_score'], min_scores[x['essay_set']-1], max_scores[x['essay_set']-1])), axis=1)
data['essay'] = data['essay'].apply(lambda x: clean_nec(x))
data.head()

,essay_id,essay_set,essay,domain1_score,normalized_score
0,1,1,"Dear local newspaper, I think effects computer...",8,0.6
1,2,1,"Dear , I believe that using computers will ben...",9,0.7
2,3,1,"Dear, More and more people use computers, but ...",7,0.5
3,4,1,"Dear Local Newspaper, I have found that many e...",10,0.8
4,5,1,"Dear , I know having computers has a positive ...",8,0.6


In [ ]:
bsp_train, bsp_test = train_test_split(data, test_size=0.33)

In [ ]:
bsp_train

,essay_id,essay_set,essay,domain1_score,normalized_score
4079,6475,3,The cyclist had very little water. It was very...,1,0.333333
9181,15131,6,The obstacles faced by the builders of the Emp...,2,0.500000
11512,18760,7,Im going to tell you a time when I was patien...,16,0.533333
1866,3061,2,"Dear Katherine Paterson, Do you believe that c...",3,0.400000
7803,12551,5,The mood of this story is very happy because t...,2,0.500000
...,...,...,...,...,...
3500,4695,2,Have you ever went to the library and while lo...,4,0.600000
8340,13088,5,The mood of the author changed during the memo...,1,0.250000
4227,6624,3,The features of the setting make him happy bec...,1,0.333333
1173,1177,1,I agree that computers have a bad effect on pe...,6,0.400000


In [ ]:
def create_embeddings (df_train, embedding_type="sbert", max_sen=64, bert_sen_avg=True, encoding="pooled" ):
    # 
    # df_train: Training dataframe containing essay and normalized_score
    #
    # embedding_type: One of "sbert" or "bert" or "glove"
    #
    # max_sen: Maximum number of sentences - determines the size of embedding vector
    #
    # bert_sen_avg: In case of BERT embedding, there is a choice to retain word level sequence
    #               in embeddings. Defaults to sentence avarage (embedding size = Max # sentences x 768
    #               If this argument is set to False (to retain word level sequence), the embedding size = 
    #               Max # of sentences x Max # of words in sen x 768
    #
    # encoding: In case of BERT, default encodng chosen in "pooled" (last stage). The other option is
    #           "last4" which sums the embeddings from last 4 hidden states
    #
    if (embedding_type == "sbert"):
        stmodel = SentenceTransformer('all-mpnet-base-v2')
        essays = df_train['essay']
        y_gold = df_train['normalized_score']
        essay_ids = df_train['essay_id']
        for j,essay in enumerate(essays):
            print ("AS: j:", j)
            eid = essay_ids.iloc[j]
            dd2= sentence_tokenize(essay)
            dd2 += [''] * (max_sen - len(dd2))
            embeddings = stmodel.encode(dd2)
            res = {"essay_id": eid, "embedding": embeddings}
            yield res
    #elsif (embedding_type == "bert"):
    #    print ("BERT Embeddings: TBD")
    #    bert_preprocess_model = hub.KerasLayer(preprocess_url)
    #    bert_model = hub.KerasLayer(encoder_url)
    #elsif (embedding_type == "glove"):
    #    print ("Glove Embeddings: TBD")

In [ ]:
essay_ids = bsp_train['essay_id']

In [ ]:
essay_ids.iloc[4]

12551

In [ ]:
essay_ids

4079      6475
9181     15131
11512    18760
1866      3061
7803     12551
         ...  
3500      4695
8340     13088
4227      6624
1173      1177
3153      4348
Name: essay_id, Length: 8693, dtype: int64

In [ ]:
essays = bsp_train['essay']

In [ ]:
essays

4079     The cyclist had very little water. It was very...
9181     The obstacles faced by the builders of the Emp...
11512    Im going to tell you a time when I was patien...
1866     Dear Katherine Paterson, Do you believe that c...
7803     The mood of this story is very happy because t...
                               ...                        
3500     Have you ever went to the library and while lo...
8340     The mood of the author changed during the memo...
4227     The features of the setting make him happy bec...
1173     I agree that computers have a bad effect on pe...
3153     I disagree with library censorship. Every pers...
Name: essay, Length: 8693, dtype: object

In [ ]:
essays.iloc[4]

'The mood of this story is very happy because the author is describing his childhood memories and all the great times he had in his home. He says over and over that he loved his home when he was a kid and enjoyed hanging with his family. Also the mood of this story is very loving. The author talks about how he realized the true meaning of family in his house. Plus he talks about how much his parents sacrificed for him to be happy'

In [ ]:
ss = create_embeddings (df_train=bsp_train)
#print (list(ss))
for row in ss:
    print (row)
    #print ("All is Well")
    #print (res['essay_id'], res['embedding'])
#Store sentences & embeddings on disc
#with open('embeddings.pkl', "wb") as fOut:
#    pickle.dump({'sentences': sentences, 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

AS: j: 0
{'essay_id': 6475, 'embedding': array([[ 0.00268849, -0.02580052,  0.01473692, ..., -0.03856998,
        -0.02442084,  0.00206265],
       [-0.07095964, -0.084003  , -0.01221643, ..., -0.01165218,
         0.03059484,  0.00136061],
       [ 0.02458133,  0.12617484,  0.003744  , ...,  0.00265196,
         0.01834287, -0.01705291],
       ...,
       [-0.01250341,  0.06143877, -0.00673449, ..., -0.00193857,
        -0.05036448, -0.01904945],
       [-0.01250337,  0.06143874, -0.00673452, ..., -0.0019386 ,
        -0.05036445, -0.01904948],
       [-0.01250337,  0.06143874, -0.00673452, ..., -0.0019386 ,
        -0.05036445, -0.01904948]], dtype=float32)}
AS: j: 1
{'essay_id': 15131, 'embedding': array([[-0.00768432,  0.05773914, -0.00207088, ..., -0.01660518,
        -0.00609289, -0.00239806],
       [-0.00770297, -0.04742905, -0.02801847, ...,  0.018707  ,
         0.00109601, -0.00011489],
       [-0.07693131,  0.02782813,  0.00244557, ..., -0.03284601,
         0.00054927, -0

KeyboardInterrupt: ignored